In [ ]:
import numpy as np
import pandas as pd
import os
import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow import keras
import time

In [ ]:
os.chdir("/kaggle/input/shopee-title-translation/")
shopee_data = pd.concat([pd.read_csv("dev_tcn.csv").drop(columns = ["split"]), pd.read_csv("dev_en.csv")], axis = 1)

In [ ]:
shopee_data

In [ ]:
SEQ_LEN = 107

In [ ]:
# ch_tokenizer = tf.keras.preprocessing.text.Tokenizer(oov_token = "<OOV>")
#ch_tokenizer.fit_on_texts(shopee_data.text)
ch_tokenizer = tfds.features.text.SubwordTextEncoder.build_from_corpus(shopee_data.text, target_vocab_size=2e15)

In [ ]:
# en_tokenizer = tf.keras.preprocessing.text.Tokenizer(oov_token = "<OOV>")
# en_tokenizer.fit_on_texts(shopee_data.translation_output)
en_tokenizer = tfds.features.text.SubwordTextEncoder.build_from_corpus(shopee_data.translation_output, target_vocab_size=2e15)

In [ ]:
# shopee_data.text = ch_tokenizer.texts_to_sequences(shopee_data.text)
# shopee_data.translation_output = en_tokenizer.texts_to_sequences(shopee_data.translation_output)
shopee_data.text = shopee_data.text.apply(ch_tokenizer.encode)
shopee_data.translation_output = shopee_data.translation_output.apply(en_tokenizer.encode)

In [ ]:
#adding start and end tokens
shopee_data.translation_output = shopee_data.translation_output.apply(lambda en : [en_tokenizer.vocab_size] + en + [en_tokenizer.vocab_size + 1])
shopee_data.text = shopee_data.text.apply(lambda ch : [ch_tokenizer.vocab_size] + ch + [ch_tokenizer.vocab_size + 1])

In [ ]:
shopee_data.text = pd.Series(list(tf.keras.preprocessing.sequence.pad_sequences(shopee_data.text, maxlen = 30, padding = "post")))
shopee_data.translation_output = pd.Series(list(tf.keras.preprocessing.sequence.pad_sequences(shopee_data.translation_output, maxlen = 30, padding = "post")))

In [ ]:
#convert to tf dataset to use with the code in the model (the model expects a tf dataset)
#its possible to modify the model to expect pandas dataframe instead, but that's a lot more work than converting the data to tf dataset
    #for now
    
tf_shopee_data = tf.data.Dataset.from_tensor_slices((list(shopee_data.text), list(shopee_data.translation_output)))
tf_shopee_data = tf_shopee_data.cache() #so loading the data is quicker
tf_shopee_data = tf_shopee_data.shuffle(100).padded_batch(50)

In [ ]:
#positional encoding
#code from https://www.tensorflow.org/tutorials/text/transformer#positional_encoding
#added to give the model context of where each word is in the sentence, since the model doesnt have convolutions or lstm

#Signature: 2DArray, 2DArray, Num
#Effects: return an n x m matrix, where n is the number of pos elements, and m is the dimension of each pos element
def get_angles(pos, i, model_depth):
    angle_rates = 1 / np.power(10000, (2 * (i//2)) / np.float32(model_depth))
    return pos * angle_rates

#Signature: Num, Num
            #total num words, num of layers in model
#Effects: Returns a modified result of get_angles to reflect positional encoding
def positional_encoding(position, model_depth):
    angle_rads = get_angles(np.arange(position)[:, np.newaxis],
                          np.arange(d_model)[np.newaxis, :],
                          model_depth)

    # apply sin to even indices in the array; 2i
    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])

    # apply cos to odd indices in the array; 2i+1
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])

    pos_encoding = angle_rads[np.newaxis, ...]

    return tf.cast(pos_encoding, dtype=tf.float32)


In [ ]:
#Masking
#Code from https://www.tensorflow.org/tutorials/text/transformer#masking

#Mask all padded tokens so that the model doesn't read the extra tokens as input

#Signature: listOfNum (listOfTokens)
#Effects: Produces 1 if the token in the sequence is a padding token, 0 otherwise
def create_padding_mask(seq):
    seq = tf.cast(tf.math.equal(seq, 0), tf.float32)

    # add extra dimensions to add the padding
    # to the attention logits.
    return seq[:, tf.newaxis, tf.newaxis, :]  # (batch_size, 1, 1, seq_len)

#Signature: Num 
#Effects: Produces a size x size matrix where each sequential array unmasks the next word in the sequence
def create_look_ahead_mask(size):
    mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
    return mask  # (seq_len, seq_len)


In [ ]:
create_padding_mask(tf.convert_to_tensor(list(shopee_data.text)))

In [ ]:
create_look_ahead_mask(5)

In [ ]:
#Dot Product Attention
#Code from https://www.tensorflow.org/tutorials/text/transformer#scaled_dot_product_attention

#Takes in 3 inputs: Query, Key, Value and Mask. Q, K, V are calculated by passing the input separatly through wq,wk,wv - Dense layers in
    #multiheaded attention. Mask refers to the 2 functions defined above.
    
#K and Q are used to calculate the Attention Weights. The Attention Weights matrix-multiplied with V to then produce the output
#Since softmax normalization is done on K, the value of K decides the "importance" given to Q
#In short, the attention weights calculate what words are most important in the sentence - multiplied by V, this ensures the output keeps
    #important words as is, and flushes out irrelevant words
    
#This particular dot product is scaled by the depth of the model because the deeper the model, the larger the variance of the matmul. To
    #bring this variance back down, we scale the results down by the model depth. We want to scale it down because if the value of the matmul
    #is very large, smaller gradients in the softmax essentially become irrelevant, and we get a very hard softmax. Scaling values down 
    #produces a much gentler softmax, which is what we want.
    
#The value of the masks is multiplied by a very large negative number so that the tokens we want to mask (represented by 1 in the sequence)
    #becomes very negative, which then produces a value close to 0 after the softmax is applied (and thus attention will not be paid to 
    #these tokens, which is what we want :D)

def scaled_dot_product_attention(q, k, v, mask):
    """Calculate the attention weights.
    q, k, v must have matching leading dimensions.
    k, v must have matching penultimate dimension, i.e.: seq_len_k = seq_len_v.
    The mask has different shapes depending on its type(padding or look ahead) 
    but it must be broadcastable for addition.

    Args:
    q: query shape == (..., seq_len_q, depth)
    k: key shape == (..., seq_len_k, depth)
    v: value shape == (..., seq_len_v, depth_v)
    mask: Float tensor with shape broadcastable 
          to (..., seq_len_q, seq_len_k). Defaults to None.

    Returns:
    output, attention_weights
    """

    matmul_qk = tf.matmul(q, k, transpose_b=True)  # (..., seq_len_q, seq_len_k)

    # scale matmul_qk
    dk = tf.cast(tf.shape(k)[-1], tf.float32)
    scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)

    # add the mask to the scaled tensor.
    if mask is not None:
        scaled_attention_logits += (mask * -1e9)  

    # softmax is normalized on the last axis (seq_len_k) so that the scores
    # add up to 1.
    attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)  # (..., seq_len_q, seq_len_k)

    output = tf.matmul(attention_weights, v)  # (..., seq_len_q, depth_v)

    return output, attention_weights

In [ ]:
#Multihead Attention
#https://www.tensorflow.org/tutorials/text/transformer#multi-head_attention

#Each multihead attention receives inputs Q V K (All calculated from the original sequence using Dense Layers wq, wv, wk)
    #The Multihead attention layer receives the input sequence 3 times to calculate q v k

#Q V K is then split into heads, and each set of heads (set of Q V K) is passed through a dot product attention layer
#The outputs of the attention layers (outputs of the heads) are then concatenated together, and passed through a final Dense layer
#The output of the dense layer is passed through to the next multihead attention layer to be split into q v k again
    
#Multihead attention consists of 4 parts:
    #Linear Layers containing wq wv wk, and afterwards split into heads
    #scaled dot product attention layer
    #concatenation of heads
    #Final Dense Layer
    
#In Code, if we want to split into heads (e.g, split Q into heads), we would reshape Q into multiple sub-arrays and pass the entire
    #masive array through the attention layer - python allows the dot product to attend to each sub-array individually. Afterwards, to
    #concatenate the arrays back, we use transpose and reshape
class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model

        assert d_model % self.num_heads == 0

        self.depth = d_model // self.num_heads

        self.wq = tf.keras.layers.Dense(d_model)
        self.wk = tf.keras.layers.Dense(d_model)
        self.wv = tf.keras.layers.Dense(d_model)

        self.dense = tf.keras.layers.Dense(d_model)
        
    def split_heads(self, x, batch_size):
        """Split the last dimension into (num_heads, depth).
        Transpose the result such that the shape is (batch_size, num_heads, seq_len, depth)
        """
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(x, perm=[0, 2, 1, 3])
    
    def call(self, v, k, q, mask):
        batch_size = tf.shape(q)[0]

        q = self.wq(q)  # (batch_size, seq_len, d_model)
        k = self.wk(k)  # (batch_size, seq_len, d_model)
        v = self.wv(v)  # (batch_size, seq_len, d_model)

        q = self.split_heads(q, batch_size)  # (batch_size, num_heads, seq_len_q, depth)
        k = self.split_heads(k, batch_size)  # (batch_size, num_heads, seq_len_k, depth)
        v = self.split_heads(v, batch_size)  # (batch_size, num_heads, seq_len_v, depth)

        # scaled_attention.shape == (batch_size, num_heads, seq_len_q, depth)
        # attention_weights.shape == (batch_size, num_heads, seq_len_q, seq_len_k)
        scaled_attention, attention_weights = scaled_dot_product_attention(
            q, k, v, mask)

        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])  # (batch_size, seq_len_q, num_heads, depth)

        concat_attention = tf.reshape(scaled_attention, 
                                      (batch_size, -1, self.d_model))  # (batch_size, seq_len_q, d_model)

        output = self.dense(concat_attention)  # (batch_size, seq_len_q, d_model)

        return output, attention_weights


In [ ]:
#Pointwise Feed Forward NN
#a simple 2 layer NN, with Relu activation in between
#part of the Encoder and Decoder Architecture
#https://www.tensorflow.org/tutorials/text/transformer#point_wise_feed_forward_network

def point_wise_feed_forward_network(d_model, dff):
    return tf.keras.Sequential([
      tf.keras.layers.Dense(dff, activation='relu'),  # (batch_size, seq_len, dff)
      tf.keras.layers.Dense(d_model)  # (batch_size, seq_len, d_model)
    ])


In [ ]:
#Encoder Layer
# https://www.tensorflow.org/tutorials/text/transformer#encoder_layer

#Each encoder layer contains the following sublayers:
    #Multihead Attention (with padding mask)
    #residual connection from before Multihead Attention Layer + Normalization
    
    #Feedforward NN
    #residual connection from before Multihead Attention Layer + Normalization
    
#Note (for decoder layers as well): Normalization is done after the residual connection is added
#The Encoder has N of these encoder layers


class EncoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, dff, rate=0.1):
        super(EncoderLayer, self).__init__()

        self.mha = MultiHeadAttention(d_model, num_heads)
        self.ffn = point_wise_feed_forward_network(d_model, dff)

        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)

        #dropout layers aren't necessary per se, but they're good for the NN (like residual connections). They randomly set outgoing
            #edges of hidden nodes to 0 during training phase to reduce the chance of overfitting
    def call(self, x, training, mask):

        attn_output, _ = self.mha(x, x, x, mask)  # (batch_size, input_seq_len, d_model)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(x + attn_output)  # (batch_size, input_seq_len, d_model)

        ffn_output = self.ffn(out1)  # (batch_size, input_seq_len, d_model)
        ffn_output = self.dropout2(ffn_output, training=training)
        out2 = self.layernorm2(out1 + ffn_output)  # (batch_size, input_seq_len, d_model)

        return out2


In [ ]:
#Decoder layer
#https://www.tensorflow.org/tutorials/text/transformer#decoder_layer

#Decoder layers have the following architecture:
    #masked multiheaded attention (both padding and lookahead mask)
    #Normalization + Residual Connection
    
    #multiheaded attention (padding mask): V and K are the ENCODER's output, while Q is the output from the first masked multihead
        #attention layer after normalization. (Note: Encoder Output, not the attention weights)
    #Normalization + Residual Connection
    
    #Feedforward network
    #Normalization + Residual Connection
    
#The Decoder has N of these sublayers

#Since the value of K determines the importance of Q (from dot product attention), The decoder is essentially looking at the Encoder's
    #output (K) to determine the importance of it's own self-attended representation of the input (Q) to predict the next word.
    
class DecoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, dff, rate=0.1):
        super(DecoderLayer, self).__init__()

        self.mha1 = MultiHeadAttention(d_model, num_heads)
        self.mha2 = MultiHeadAttention(d_model, num_heads)

        self.ffn = point_wise_feed_forward_network(d_model, dff)

        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm3 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)
        self.dropout3 = tf.keras.layers.Dropout(rate)


    def call(self, x, enc_output, training, 
           look_ahead_mask, padding_mask):
        # enc_output.shape == (batch_size, input_seq_len, d_model)

        attn1, attn_weights_block1 = self.mha1(x, x, x, look_ahead_mask)  # (batch_size, target_seq_len, d_model)
        attn1 = self.dropout1(attn1, training=training)
        out1 = self.layernorm1(attn1 + x)

        attn2, attn_weights_block2 = self.mha2(
            enc_output, enc_output, out1, padding_mask)  # (batch_size, target_seq_len, d_model)
        attn2 = self.dropout2(attn2, training=training)
        out2 = self.layernorm2(attn2 + out1)  # (batch_size, target_seq_len, d_model)

        ffn_output = self.ffn(out2)  # (batch_size, target_seq_len, d_model)
        ffn_output = self.dropout3(ffn_output, training=training)
        out3 = self.layernorm3(ffn_output + out2)  # (batch_size, target_seq_len, d_model)

        return out3, attn_weights_block1, attn_weights_block2


In [ ]:
#Encoder
#https://www.tensorflow.org/tutorials/text/transformer#encoder

#Architecture:
    #Input Embedding
    #Positional Encoding
    # N x Encoder layers

#The input is embedded and then summed with the positional encoding, and then passed onto the N Encoder layers
class Encoder(tf.keras.layers.Layer):
    def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size,
               maximum_position_encoding, rate=0.1):
        super(Encoder, self).__init__()

        self.d_model = d_model
        self.num_layers = num_layers

        self.embedding = tf.keras.layers.Embedding(input_vocab_size, d_model)
        self.pos_encoding = positional_encoding(maximum_position_encoding, 
                                                self.d_model)


        self.enc_layers = [EncoderLayer(d_model, num_heads, dff, rate) 
                           for _ in range(num_layers)]

        self.dropout = tf.keras.layers.Dropout(rate)

    def call(self, x, training, mask):

        seq_len = tf.shape(x)[1]

        # adding embedding and position encoding.
        x = self.embedding(x)  # (batch_size, input_seq_len, d_model)
        x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        x += self.pos_encoding[:, :seq_len, :]

        x = self.dropout(x, training=training)

        for i in range(self.num_layers):
            x = self.enc_layers[i](x, training, mask)

        return x  # (batch_size, input_seq_len, d_model)


In [ ]:
#Decoder
#https://www.tensorflow.org/tutorials/text/transformer#decoder

#Architecture:
    #Output Embedding (For cases where we're predicting, we only embed the start token. For training purposes, we may want to do
        #teacher-forcing, where we embed the correct result for the next cycle, regardless of whether the model got it right or not)
    #Positional Encoding (Again, for predicting, only positionally encode the start token)
    #N x Decoder Layers
    
#The output of the final decoder layer is then passed through a final linear layer to predict what the next word is

#The way the model (specifically the Decoder section) will train is as follows:
    #The first token the decoder receives will be the start token
    #afterwards, if the model is still training, we will concatenate the actual next word into the sentence and have the model predict
        # the word. If the model isn't training, the model will concatenate its own prediction into the sentence and continue predicting
        #the next word
    #the above process is repeated until the model finishes translating the sentence
class Decoder(tf.keras.layers.Layer):
    def __init__(self, num_layers, d_model, num_heads, dff, target_vocab_size,
               maximum_position_encoding, rate=0.1):
        super(Decoder, self).__init__()

        self.d_model = d_model
        self.num_layers = num_layers

        self.embedding = tf.keras.layers.Embedding(target_vocab_size, d_model)
        self.pos_encoding = positional_encoding(maximum_position_encoding, d_model)

        self.dec_layers = [DecoderLayer(d_model, num_heads, dff, rate) 
                           for _ in range(num_layers)]
        self.dropout = tf.keras.layers.Dropout(rate)

    def call(self, x, enc_output, training, 
           look_ahead_mask, padding_mask):

        seq_len = tf.shape(x)[1]
        attention_weights = {}

        x = self.embedding(x)  # (batch_size, target_seq_len, d_model)
        x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        x += self.pos_encoding[:, :seq_len, :]

        x = self.dropout(x, training=training)

        for i in range(self.num_layers):
            x, block1, block2 = self.dec_layers[i](x, enc_output, training,
                                                 look_ahead_mask, padding_mask)

            attention_weights['decoder_layer{}_block1'.format(i+1)] = block1
            attention_weights['decoder_layer{}_block2'.format(i+1)] = block2

        # x.shape == (batch_size, target_seq_len, d_model)
        return x, attention_weights

    

In [ ]:
#Creating the full transformer
#https://www.tensorflow.org/tutorials/text/transformer#create_the_transformer
#Architecture: Encode, Decoder, Final Linear Layer to output predictions

# pe_input is positional encoding for the input, pe_output is for positional encoding of the output
# training is a boolean, True if the model is training, false otherwise. Used for the dropout layer and to enforce teacher-forcing

class Transformer(tf.keras.Model):
    def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size, 
               target_vocab_size, pe_input, pe_target, rate=0.1):
        super(Transformer, self).__init__()

        self.encoder = Encoder(num_layers, d_model, num_heads, dff, 
                               input_vocab_size, pe_input, rate)

        self.decoder = Decoder(num_layers, d_model, num_heads, dff, 
                               target_vocab_size, pe_target, rate)

        self.final_layer = tf.keras.layers.Dense(target_vocab_size)
    
    def call(self, inp, tar, training, enc_padding_mask, 
           look_ahead_mask, dec_padding_mask):

        enc_output = self.encoder(inp, training, enc_padding_mask)  # (batch_size, inp_seq_len, d_model)

        # dec_output.shape == (batch_size, tar_seq_len, d_model)
        dec_output, attention_weights = self.decoder(
            tar, enc_output, training, look_ahead_mask, dec_padding_mask)

        final_output = self.final_layer(dec_output)  # (batch_size, tar_seq_len, target_vocab_size)

        return final_output, attention_weights


In [ ]:
#Hyperparameters
num_layers = 4
d_model = 128
dff = 512
num_heads = 8

input_vocab_size = ch_tokenizer.vocab_size + 2
target_vocab_size = en_tokenizer.vocab_size + 2
dropout_rate = 0.1
EPOCHS = 120
BATCH_SIZE = 50

In [ ]:
#Custome Optimizer
#Normally I'd use a built in Adam optimizer - but I will follow the tutorial and use the optimizer with the learning rate scheduler
#https://www.tensorflow.org/tutorials/text/transformer#optimizer
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=4000):
        super(CustomSchedule, self).__init__()

        self.d_model = d_model
        self.d_model = tf.cast(self.d_model, tf.float32)

        self.warmup_steps = warmup_steps

    def __call__(self, step):
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps ** -1.5)

        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)
    
learning_rate = CustomSchedule(d_model)

optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98, 
                                     epsilon=1e-9)

In [ ]:
#Loss and Metrics
#https://www.tensorflow.org/tutorials/text/transformer#loss_and_metrics

#We need to make sure to apply a pad masking when calculating the loss since the target sequences are padded
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_sum(loss_)/tf.reduce_sum(mask)

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(
    name='train_accuracy')


In [ ]:
#training and checkpointing
# https://www.tensorflow.org/tutorials/text/transformer#training_and_checkpointing
transformer = Transformer(num_layers, d_model, num_heads, dff,
                          input_vocab_size, target_vocab_size, 
                          pe_input=input_vocab_size, 
                          pe_target=target_vocab_size,
                          rate=dropout_rate)

#This function isn't necessary per se, but it makes the code a lot cleaner. It essentially takes the masking functions defined at the start
    #(create padding mask and create look ahead mask), and applies it to the input and output, and returns the 3 types of masks required by
    #the Encoder and Decoder (encoding padding mask, decoding padding+look ahead mask, decoding padding mask)
def create_masks(inp, tar):
    # Encoder padding mask
    enc_padding_mask = create_padding_mask(inp)

    # Used in the 2nd attention block in the decoder.
    # This padding mask is used to mask the encoder outputs.
    dec_padding_mask = create_padding_mask(inp)

    # Used in the 1st attention block in the decoder.
    # It is used to pad and mask future tokens in the input received by 
    # the decoder.
    look_ahead_mask = create_look_ahead_mask(tf.shape(tar)[1])
    dec_target_padding_mask = create_padding_mask(tar)
    combined_mask = tf.maximum(dec_target_padding_mask, look_ahead_mask)

    return enc_padding_mask, combined_mask, dec_padding_mask


# Creating checkpoint paths
checkpoint_path = "/kaggle/working/"

ckpt = tf.train.Checkpoint(transformer=transformer,
                           optimizer=optimizer)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=3)

# if a checkpoint exists, restore the latest checkpoint.
if ckpt_manager.latest_checkpoint:
    ckpt.restore(ckpt_manager.latest_checkpoint)
    print ('Latest checkpoint restored!!')



# train_step_signature = [
#     tf.TensorSpec(shape=(50, 30), dtype=tf.int32),
#     tf.TensorSpec(shape=(50, 30), dtype=tf.int32),
# ]

# @tf.function(input_signature=train_step_signature)
def train_step(inp, tar):
    tar_inp = tar[:, :-1]
    tar_real = tar[:, 1:]

    enc_padding_mask, combined_mask, dec_padding_mask = create_masks(inp, tar_inp)

    with tf.GradientTape() as tape:
        print("predictions")
        predictions, _ = transformer(inp, tar_inp, 
                                     True, 
                                     enc_padding_mask, 
                                     combined_mask, 
                                     dec_padding_mask)
        
        print("loss")
        loss = loss_function(tar_real, predictions)

    print("gradient descent")
    gradients = tape.gradient(loss, transformer.trainable_variables)    
    optimizer.apply_gradients(zip(gradients, transformer.trainable_variables))

    print("metrics")
    train_loss(loss)
    train_accuracy(tar_real, predictions)

    
    
def train_transformer(tf_dataset, EPOCHS, verbose_interval):
    for epoch in range(EPOCHS):
        start = time.time()

        train_loss.reset_states()
        train_accuracy.reset_states()

        # inp -> portuguese, tar -> english
        for (batch, (inp, tar)) in enumerate(tf_dataset):
            train_step(inp, tar)

            if batch % verbose_interval == 0:
                  print ('Epoch {} Batch {} Loss {:.4f} Accuracy {:.4f}'.format(
                  epoch + 1, batch, train_loss.result(), train_accuracy.result()))

        if (epoch + 1) % 5 == 0:
            ckpt_save_path = ckpt_manager.save()
            print ('Saving checkpoint for epoch {} at {}'.format(epoch+1,
                                                                 ckpt_save_path))

        print ('Epoch {} Loss {:.4f} Accuracy {:.4f}'.format(epoch + 1, 
                                                    train_loss.result(), 
                                                    train_accuracy.result()))

        print ('Time taken for 1 epoch: {} secs\n'.format(time.time() - start))

train_transformer(tf_shopee_data, EPOCHS = EPOCHS, verbose_interval = 2)

In [ ]:
# os.chdir("/kaggle/input/chinese-english-parallel-corpus-education-thesis/")
# thesis = pd.read_fwf("Bi-Thesis.txt", header = None).iloc[:,0]
# en_thesis = thesis.iloc[[i for i in range(0,50000,2)]]
# ch_thesis = thesis.iloc[[i for i in range(1,50000,2)]]

# # en_tokenizer.build_from_corpus(en_thesis, target_vocab_size=2e15)
# # ch_tokenizer.build_from_corpus(ch_thesis, target_vocab_size=2e15)

# en_thesis = en_thesis.apply(en_tokenizer.encode)
# ch_thesis = ch_thesis.apply(ch_tokenizer.encode)

# en_thesis = en_thesis.apply(lambda en : [en_tokenizer.vocab_size] + en + [en_tokenizer.vocab_size + 1])
# en_thesis = pd.Series(list(tf.keras.preprocessing.sequence.pad_sequences(en_thesis, padding = "post")))
# ch_thesis = ch_thesis.apply(lambda ch : [ch_tokenizer.vocab_size] + ch + [ch_tokenizer.vocab_size + 1])
# ch_thesis = pd.Series(list(tf.keras.preprocessing.sequence.pad_sequences(ch_thesis, padding = "post")))

# tf_thesis = tf.data.Dataset.from_tensor_slices((list(ch_thesis), list(en_thesis)))
# tf_thesis = tf_thesis.cache()
# tf_thesis = tf_thesis.shuffle(100).padded_batch(50)

In [ ]:
#Signature: pd.Series, pd.Series => tf.data.Dataset
def preprocessor(ch_data, en_data):
    ch_data = ch_data.apply(ch_tokenizer.encode)
    ch_data = ch_data.apply(lambda ch : [ch_tokenizer.vocab_size] + ch + [ch_tokenizer.vocab_size + 1])
    ch_data = pd.Series(list(tf.keras.preprocessing.sequence.pad_sequences(ch_data, padding = "post"))) 

In [ ]:
# train_transformer(tf_thesis, EPOCHS = 10, verbose_interval = 5000)

In [ ]:
#Making predictions / evaluating
#https://www.tensorflow.org/tutorials/text/transformer#evaluate

def evaluate(inp_sentence):
    start_token = [ch_tokenizer.vocab_size]
    end_token = [ch_tokenizer.vocab_size + 1]

    #Adding start and end tokens to input sequence (in chinese)
    inp_sentence = start_token + ch_tokenizer.encode(inp_sentence) + end_token
    encoder_input = tf.expand_dims(inp_sentence, 0)

    #Add English start token to the first word of the decoder
    decoder_input = [en_tokenizer.vocab_size]
    output = tf.expand_dims(decoder_input, 0)
    
    
    #Code for the model to auto-regressively predict the sentence
    for i in range(SEQ_LEN): #Defined at the start of the notebook, the longest sequence in ch that we have to predict
        enc_padding_mask, combined_mask, dec_padding_mask = create_masks(
            encoder_input, output) #This is in the for loop because we will concatenate the prediction to the output until the end token
                                    #is reached. Since the output changes per iteration, we create new masks for it entirely.

            
        # predictions.shape == (batch_size, seq_len, vocab_size)
        predictions, attention_weights = transformer(encoder_input, 
                                                     output,
                                                     False,
                                                     enc_padding_mask,
                                                     combined_mask,
                                                     dec_padding_mask)

        # select the last word from the seq_len dimension bc we're only interested in what it think the next word will be
        predictions = predictions[: ,-1:, :]  # (batch_size, 1, vocab_size)

        #argmax finds the index position of the output the model is most sure is correct
            #axis = -1 means the last axis (the vocab_size axis)
            #the transformer is effectively looking at the entire vocab and finding the index of the output it thinks is correct. This index
                #is also the token ID
        predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)

        # If the Token ID is the end ID, return the sequence.
        if predicted_id == en_tokenizer.vocab_size+1:
            return tf.squeeze(output, axis=0), attention_weights

        #Otherwise, add the token it thinks is right to the end of the output. The output is then fed back into the Decoder
        output = tf.concat([output, predicted_id], axis=-1)

    #If somehow we haven't reached the end token by the time the max sequence runs out, just return the output (sequence of tokens) that
        #we've predicted so far
    return tf.squeeze(output, axis=0), attention_weights

def translate(sentence):
    result, attention_weights = evaluate(sentence) #Getting the preducted output tokens
  
    predicted_sentence = en_tokenizer.decode([i for i in result 
                                            if i < en_tokenizer.vocab_size])  #Decoding the output tokens into an english sentence

    print('Input: {}'.format(sentence))
    print('Predicted translation: {}'.format(predicted_sentence))
    return predicted_sentence

In [ ]:
test_tcn = pd.read_csv("test_tcn.csv").text

In [ ]:
test_tcn = test_tcn.apply(ch_tokenizer.encode)
test_tcn = test_tcn.apply(lambda ch : [ch_tokenizer.vocab_size] + ch + [ch_tokenizer.vocab_size + 1])

In [ ]:
translate(test_tcn[0])

In [ ]:
submission = pd.Series(data= np.zeros(10000),name = "product_title")
for i in range(test_tcn.shape[0]):
    print(i)
    submission[i] = translate(test_tcn[i])

In [ ]:
submission[[j for j in range(5000,10000)]] = submission[[i for i in range(5000)]]

In [ ]:
submission.to_csv("/kaggle/working/submission.csv", index=False)

In [ ]:
submission = pd.read_csv("/kaggle/working/submission.csv")
submission.name = "product_title"

In [ ]:
submission.name = "product_title"
category = pd.read_csv("test_tcn.csv").split
category.name = "category"

In [ ]:
submission = pd.concat([submission, category], axis=1)

In [ ]:
submission.columns = ["product_title", "category"]

In [ ]:
submission.to_csv("/kaggle/working/submission2.csv", index=False)

In [ ]:
submission

In [ ]:
test = pd.read_csv("/kaggle/working/submission2.csv")

In [ ]:
arr = submission[pd.DataFrame.duplicated(submission)].index
for i in arr:
    submission.loc[i] = submission.loc[i] + str(i)

In [ ]:
submission